In [1]:
import numpy as np
import pandas as pd
import random

## Conjuntos 
Del modelo de optimización

| **Término** | **Descripción**                |
|-------------|--------------------------------|
| $I$         | Conjunto de productores        |
| $J$         | Conjunto de centros de acopio  |
| $K$         | Conjunto de clientes           |
| $P$         | Conjunto de productos          |
| $T$         | Conjunto de periodos de tiempo |

In [2]:
# I = 'Productores'
J = 'CAcopios'
K = 'Clientes'
P = 'Productos'
# T = 'Tiempo'

In [3]:
csv_base = './csv/'

routes = {
    # Costo fijo del centro de acopio
    'cfa': 'CostoFijoCA.csv',
    # Costo variable del centro de acopio
    'cva' : 'CostoVarCA.csv',
    # Costo de inventario del centro de acopio
    'cia' : 'CostoInvCA.csv',
    # Costo de transporte del centro de acopio al cliente
    'ctac' : 'CostoTransCAClie.csv'
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

# pd.read_csv(csv_base + routes['cfa'], **csv_config)

## Centros de acopio

In [4]:
J_df = pd.read_csv(csv_base + J + '.csv')
J_df

,CAcopios
0,San Juan del Cesar
1,Riohacha
2,Maicao
3,Guamal
4,Pivijay
5,Nueva Granada
6,Monteria
7,Chinu
8,Planeta rica


## Clientes

In [5]:
K_df = pd.read_csv(csv_base + K + '.csv')
K_df

,Clientes
0,Distribuidor1
1,Distribuidor2
2,Distribuidor3
3,Tienda1
4,Tienda2
5,Mercado1
6,Mercado2
7,Mercado3
8,Panaderia1
9,Panaderia2


## Productos

In [6]:
P_df = pd.read_csv(csv_base + P + '.csv')
P_df.head()

,Productos
0,Queso duro
1,Queso blando


# Escenarios de prueba
Tomando demandas por cada centro de acopio, se definen distintos escenarios de posibles casos.

**Escenario 1**: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

**Escenario 2**: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

**Escenario 3**: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Demandas
Generando demandas para cada cliente, se usarán como entrada para el modelo

In [7]:
K_df = pd.read_csv(csv_base + K + '.csv')
clientes = K_df[K].drop_duplicates()

demandas = []

for client in clientes:
    demandas.append(20 * random.randint(0, 100))

clientes = pd.DataFrame(clientes)

clientes['Demanda'] = pd.DataFrame(demandas)
clientes

,Clientes,Demanda
0,Distribuidor1,340
1,Distribuidor2,340
2,Distribuidor3,900
3,Tienda1,1040
4,Tienda2,1780
5,Mercado1,60
6,Mercado2,1320
7,Mercado3,680
8,Panaderia1,2000
9,Panaderia2,680


### Capacidades 
Obteniendo las capacidades de cada centro de acopio

In [8]:
J_df = pd.read_csv(csv_base + J + '.csv')
cacopios = J_df['CAcopios']
capacidades = []

for c_acopio in cacopios:
    capacidades.append(10 * random.randint(0, 100))

cacopios = pd.DataFrame(cacopios)
cacopios['Capacidad'] = pd.DataFrame(capacidades)
cacopios

,CAcopios,Capacidad
0,San Juan del Cesar,540
1,Riohacha,750
2,Maicao,270
3,Guamal,860
4,Pivijay,430
5,Nueva Granada,10
6,Monteria,820
7,Chinu,210
8,Planeta rica,70


## Función objetivo
$$
Min(F) = CProduccion_t + COperacion_t + CInventario_t + CTransporte_t
$$

#### Costos de producción
$$
  CProduccion_t = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} CostoProduccion_{pit} PA_{pijt} \qquad \forall_t \in T
$$

In [9]:
# Costo producción
# cp_df = pd.read_csv(csv_base + routes['cp'], **csv_config)
# cp_df.head()

#### Costos de Operación
$$
  COperacion_t = CFijos_t + CVariables_t
$$

In [10]:
cfa_df = pd.read_csv(csv_base + routes['cfa'], **csv_config)
cfa_df.head()

,CAcopios,Productos,CostoFijoCA
0,San Juan del Cesar,Queso duro,22000
1,San Juan del Cesar,Queso blando,25000
2,Riohacha,Queso duro,30000
3,Riohacha,Queso blando,31000
4,Maicao,Queso duro,28000


In [11]:
# Costo variable
# cva_df = pd.read_csv(csv_base + routes['cva'], **csv_config)
# cva_df

#### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [12]:
cia_df = pd.read_csv(csv_base + routes['cia'], **csv_config)
cia_df.head()

,CAcopios,Productos,CostoInvCA
0,San Juan del Cesar,Queso duro,1000
1,San Juan del Cesar,Queso blando,1500
2,Riohacha,Queso duro,500
3,Riohacha,Queso blando,800
4,Maicao,Queso duro,900


#### Costos de Transporte
$$
  CTransporte_t = \sum_{j \in J} \sum_{i \in I} \sum_{p \in P} CostoTransAcopio_{pijt} PA_{pijt} + \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} CostoTransAcopioClie_{pjkt} \qquad \forall_t \in T
$$

In [13]:
# Costo productor-acopio
# ctpa_df = pd.read_csv(csv_base + routes['ctpa'], **csv_config)
# ctpa_df

In [14]:
ctac_df = pd.read_csv(csv_base + routes['ctac'], **csv_config)
ctac_df.head()

,CAcopios,Clientes,CostoTransCAClie
0,San Juan del Cesar,Distribuidor1,68000
1,San Juan del Cesar,Distribuidor2,38000
2,San Juan del Cesar,Distribuidor3,56000
3,San Juan del Cesar,Tienda1,32000
4,San Juan del Cesar,Tienda2,95000


## Esquema de representación
Se recibe la demanda $DemandaClie_{pkt}$ como parámetro de entrada.

Se representa la cantidad $n$ de centros de acopio como un vector, donde se asigna una cantidad $x_i$ para cada centro de acopio $J_i$, la cantidad asignada representa la demanda que va a ser suplida por ese centro de acopio.
$$
\begin{array} {|r|r|r|r|r|r|}
    \hline x_0 & x_1 & x_2 & x_3 & \cdots & x_n \\
    \hline
\end{array}
\quad \therefore \quad x_i = AC_{pjkt}
$$
Donde se aplican las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{n} x_i &= DemandaClie_{pkt} \\
    x_i &\leq InvCA_{pjt}
\end{align*}
$$

In [15]:
X = np.zeros(J_df.shape)
dem = clientes.loc[0, 'Demanda']

for i in range(X.shape[0]):
    idx = random.randint(0, dem)
    cap = cacopios.loc[i, 'Capacidad']
        
    if i == X.size - 1 and dem != 0:
        X[i] = dem
        dem -= 0
    else:
        if idx > cap:
            X[i] = cap
            dem -= cap
        else:
            X[i] = idx
            dem -= idx
    
print(X)
print(f'Suma de la distribución: {X.sum()}')

[[204.]
 [  2.]
 [118.]
 [ 13.]
 [  0.]
 [  0.]
 [  1.]
 [  0.]
 [  2.]]
Suma de la distribución: 340.0


In [17]:
for idx in range(X.shape[0]):
    if X[idx] == 0:
        continue
    
    ca = J_df.loc[idx]
    
    costo_fijo_ca = cfa_df[(cfa_df[J] == ca[J]) & (cfa_df[P] == 'Queso duro')]
    costo_inv_ca = cia_df[(cia_df[J] == ca[J]) & (cia_df[P] == 'Queso duro')]
    costo_trans_ca = ctac_df[(ctac_df[J] == ca[J]) & (ctac_df[K] == 'Distribuidor1')]

    delta = X[idx] + costo_fijo_ca['CostoFijoCA']
    delta += X[idx] * costo_inv_ca['CostoInvCA']
    delta += np.array(costo_trans_ca['CostoTransCAClie'])
    
    print(delta)
    
    # temp = X[idx] 

0    294204.0
Name: CostoFijoCA, dtype: float64
2    52002.0
Name: CostoFijoCA, dtype: float64
4    233318.0
Name: CostoFijoCA, dtype: float64
6    54313.0
Name: CostoFijoCA, dtype: float64
12    85001.0
Name: CostoFijoCA, dtype: float64
16    75402.0
Name: CostoFijoCA, dtype: float64


## scipy

In [18]:
from scipy.optimize import dual_annealing

def f(x):
    return x

bounds = [(0, X.size)]
result = dual_annealing(f, bounds=bounds)
print(result)

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: 0.0
       x: [ 0.000e+00]
     nit: 1000
    nfev: 2005
    njev: 2
    nhev: 0


In [ ]:
# dc_df = pd.read_csv(r'' + csv_base + routes.get('demanda_clie'))
# res = dc_df.groupby('Clientes')['DemandaClie'].agg('sum')